# Data cleansing

In [15]:
# Author - Abishek Murali
# Written - December 2017
# This script reads data from the raw data file, cleanses the data, creates a codebook and then 
# writes the codebook and cleansed data into a csv
# 1. Raw data file to retrieve data and 
# 2. Destination of codebook file
# 3. Destination of cleansed data file

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Prepare codebook

In [17]:
data = pd.read_csv("../data/raw_data.csv", encoding='latin-1')
df = data.copy()
df.columns = ['C'+ str(i + 1) for i in range(len(df.columns))] #encode column names with prefix 'C' followed by number
d = zip(data.columns,df.columns)
pd.options.display.max_rows = 65
codebook = pd.DataFrame([dict(d)]).T.reset_index().rename(columns = {"index":"Old_names",0:"New_names"})

if os.path.exists("../docs/Codebook.csv"): # Write into codebook
    os.remove("../docs/Codebook.csv")
    
codebook.sort_values(by = ['New_names']).to_csv("../docs/Codebook.csv",index=False)

In [18]:
df.columns = ['C'+ str(i + 1) for i in range(len(data.columns))]

In [19]:
cb = pd.read_csv("../docs/Codebook.csv", encoding='latin-1')

# Cleansing data

### Classifying gender into 3 groups for ease of analysis - Male, Female and NB

In [20]:
#df.C58.unique()

In [21]:
# clean the genders by grouping the genders into 3 categories: Female, Male, Genderqueer/Other
# sourced from https://www.kaggle.com/jchen2186/data-visualization-with-python-seaborn

df['C58'] = df['C58'].replace([
    'male', 'Male ', 'M', 'm', 'man', 'Cis male',
    'Male.', 'Male (cis)', 'Man', 'Sex is male',
    'cis male', 'Malr', 'Dude', "I'm a man why didn't you make this a drop down question. You should of asked sex? And I would of answered yes please. Seriously how much text can this take? ",
    'mail', 'M|', 'male ', 'Cis Male', 'Male (trans, FtM)',
    'cisdude', 'cis man', 'MALE'], 'Male')
df['C58'] = df['C58'].replace([
    'female', 'I identify as female.', 'female ',
    'Female assigned at birth ', 'F', 'Woman', 'fm', 'f',
    'Cis female', 'Transitioned, M2F', 'Female or Multi-Gender Femme',
    'Female ', 'woman', 'female/woman', 'Cisgender Female', 
    'mtf', 'fem', 'Female (props for making this a freeform field, though)',
    ' Female', 'Cis-woman', 'AFAB', 'Transgender woman',
    'Cis female '], 'Female')
df['C58'] = df['C58'].replace([
    'Bigender', 'non-binary,', 'Genderfluid (born female)',
    'Other/Transfeminine', 'Androgynous', 'male 9:1 female, roughly',
    'nb masculine', 'genderqueer', 'Human', 'Genderfluid',
    'Enby', 'genderqueer woman', 'Queer', 'Agender', 'Fluid',
    'Genderflux demi-girl', 'female-bodied; no feelings about gender',
    'non-binary', 'Male/genderqueer', 'Nonbinary', 'Other', 'none of your business',
    'Unicorn', 'human', 'Genderqueer'], 'Non-binary')

# replace the one null with Male, the mode gender, so we don't have to drop the row
df['C58'] = df['C58'].replace(np.NaN, 'Male')

### Removing self employed professionals from analysis for tech companies

In [22]:
#Removing self employed professionals for the sake of this analysis
df = df.query('C2 == 0')

In [23]:
a = df.applymap(np.isreal).all()

In [24]:
# df.iloc[:,-np.where(a)[0]].head()

# Summary tables

In [25]:
string_sum = df.describe(include = [np.object])
if os.path.exists("../results/Summary_strings.csv"): # Write into string summary table
    os.remove("../results/Summary_strings.csv")
    
string_sum.to_csv("../results/Summary_strings.csv")

In [26]:
string_sum = df.describe(include = [np.object])
if os.path.exists("../results/Summary_numeric.csv"): # Write into numeric summary table
    os.remove("../results/Summary_numeric.csv")
    
string_sum.to_csv("../results/Summary_numeric.csv")

# Write cleansed data to file

In [27]:
df.to_csv("../data/cleansed_data.csv")